In [2]:
# !wget https://huggingface.co/datasets/mesolitica/malaysian-benchmark/resolve/main/iium-coffession-en.json

In [4]:
from tqdm import tqdm
import requests
import os
import json

In [6]:
with open('iium-coffession-en.json') as fopen:
    d = json.load(fopen)
    
len(d)

1000

In [7]:
d[0]

{'from': "Masalah jerawat akan menjadi lebih teruk kalau guna pencuci muka bersifat alkali/high pH, dan bila guna pencuci muka yang low pH, acne akan berkurangan dan kulit akan improve (Kalau u all rajin, boleh la baca study Korting et al, tajuk dia 'The Influence of the Regular Use of a Soap or an Acidic Syndet Bar on Pre-Acne', tak pasti jumpa ke tak kalau Google, tapi dalam blog yang I baca ni ada direct link pergi ke pdf file study tu).",
 'to': "Acne problems can be worse if you use alkaline / high pH cleanser, and when you use low pH cleanser, acne will decrease and your skin will improve (If you are all diligent, you can read the study Korting et al. Influence of the Regular Use of a Soap or an Acidic Syndet Bar on Pre-Acne ', not sure if you can find it through Google, but in the blog I read there is a direct link to the pdf file study)."}

In [5]:
!mkdir google-translate-iium-en

In [9]:
for i in tqdm(range(len(d))):
    filename = os.path.join('google-translate-iium-en', f'{i}.json')
    
    if os.path.exists(filename):
        continue
    
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
    }
    
    l = d[i]['from']

    json_data = {
        'text': l,
    }

    response = requests.post('http://localhost:8003/translate', headers=headers, json=json_data)
    r = response.json()
    with open(filename, 'w') as fopen:
        json.dump({'text': l, 'r': r}, fopen)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [45:52<00:00,  2.75s/it]


In [10]:
from sacrebleu.metrics import BLEU, CHRF, TER

chrf = CHRF(word_order = 2)

In [15]:
predicted = []
for i in range(len(d)):
    filename = os.path.join('google-translate-iium-en', f'{i}.json')
    with open(filename) as fopen:
        d_ = json.load(fopen)

    predicted.append(d_['r']['result'])

In [17]:
score = chrf.corpus_score(predicted, [[d_['to'] for d_ in d]])
score

chrF2++ = 53.83